# 🚗 Sprint Challenge 4 – Previsão de Acidentes com LSTMs
## Case Sompo: Antecipando Padrões de Risco em Rodovias Brasileiras

---

**Equipe Big 5**
- Lucca Phelipe Masini - RM 564121
- Luiz Henrique Poss - RM 562177
- Luis Fernando de Oliveira Salgado - RM 561401
- Igor Paixão Sarak - RM 563726
- Bernardo Braga Perobeli - RM 562468

---

## 🎯 Target Escolhido: Prever Número Total de Acidentes

**Objetivo**: Prever o **número total de acidentes** que ocorrerão em cada semana por estado.

- **Target**: Contagem de acidentes (valores inteiros)
- **Interpretação**: Quantos acidentes esperamos na próxima semana
- **Aplicação prática**: Alocação de recursos (patrulhas, ambulâncias)

### Justificativa

Escolhemos **prever número de acidentes** porque:

1. ✅ **Padrões temporais claros** - Variação dia útil vs fim de semana, feriados, sazonalidade
2. ✅ **Features históricas são preditivas** - Número de acidentes nas últimas semanas ajuda a prever a próxima
3. ✅ **LSTM ideal para séries temporais** - Captura dependências de longo prazo
4. ✅ **Métricas intuitivas** - MAE mostra erro médio em número de acidentes (ex: ±5 acidentes/semana)
5. ✅ **Valor prático real** - Gestores podem planejar operações baseado em volume esperado

---

## 📚 Passo 1: Instalação e Importação de Bibliotecas

Primeiro, vamos instalar e importar todas as bibliotecas necessárias para o projeto.


In [ ]:
!pip install openpyxl --quiet
print("✅ Bibliotecas instaladas!")

---

## 📥 Passo 2: Carregamento dos Dados

Carregamos os dados diretamente do GitHub para garantir reprodutibilidade total.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import urllib.request
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("📚 Bibliotecas importadas!")

---

## 🎯 Passo 3: Pré-processamento e Criação da Variável Target

Criamos a variável binária `severo` que identifica acidentes com mortos ou feridos graves.


In [ ]:
print("📥 Baixando dataset da PRF do GitHub...")

github_raw_url = 'https://raw.githubusercontent.com/9luis7/lstm-acidentes-prf/main/dados/datatran2025.xlsx'
output_filename = 'dados_acidentes.xlsx'

try:
    urllib.request.urlretrieve(github_raw_url, output_filename)
    df = pd.read_excel(output_filename)
    print(f"✅ Dataset carregado: {len(df):,} acidentes")
    print("\n📊 Período:", df['data_inversa'].min(), "até", df['data_inversa'].max())
    print("📊 Estados:", df['uf'].nunique(), "UFs")
except Exception as e:
    print(f"❌ Erro: {e}")
    raise

---

## 🔄 Passo 4: Agregação Semanal

Transformamos acidentes individuais em séries temporais semanais por estado.


In [ ]:
print("🎯 Criando variável target 'severo'...")

df['horario'] = pd.to_datetime(df['horario'], format='%H:%M:%S', errors='coerce').dt.time
df['severo'] = ((df['mortos'] > 0) | (df['feridos_graves'] > 0)).astype(int)

colunas_relevantes = ['data_inversa', 'horario', 'uf', 'br', 'km', 'pessoas', 'veiculos', 'severo']
df_limpo = df[colunas_relevantes].copy()
df_limpo['horario'].fillna(pd.to_datetime('12:00:00').time(), inplace=True)

print("✅ Variável 'severo' criada!")
print("\n📊 Distribuição:")
print(df_limpo['severo'].value_counts(normalize=True))

---

## 🎨 Passo 5: Feature Engineering

Criamos 12 features enriquecidas: temporais, sazonalidade e histórico (lags).


In [ ]:
print("🔄 Agregando dados em séries temporais semanais...")

df_indexed = df_limpo.set_index('data_inversa')

weekly_df = df_indexed.groupby([pd.Grouper(freq='W'), 'uf']).agg(
    total_acidentes=('severo', 'count'),
    acidentes_severos=('severo', 'sum'),
    pessoas_total=('pessoas', 'sum'),
    veiculos_total=('veiculos', 'sum'),
    pessoas_media=('pessoas', 'mean'),
    veiculos_media=('veiculos', 'mean')
).reset_index()

weekly_df['prop_severos'] = np.where(
    weekly_df['total_acidentes'] > 0,
    weekly_df['acidentes_severos'] / weekly_df['total_acidentes'],
    0
)

print(f"✅ Dados agregados: {len(weekly_df):,} semanas × estados")

---

## 🔢 Passo 6: Criação de Sequências Temporais

Criamos janelas de 8 semanas para prever a semana seguinte. Normalizamos os dados com MinMaxScaler.


In [ ]:
print("🎨 Criando features temporais e de histórico...")

# Temporais
weekly_df['dia_semana'] = weekly_df['data_inversa'].dt.dayofweek
weekly_df['mes'] = weekly_df['data_inversa'].dt.month
weekly_df['fim_semana'] = weekly_df['dia_semana'].isin([5, 6]).astype(int)

# Sazonalidade
weekly_df['sazonalidade_sen'] = np.sin(2 * np.pi * weekly_df['data_inversa'].dt.dayofyear / 365)
weekly_df['sazonalidade_cos'] = np.cos(2 * np.pi * weekly_df['data_inversa'].dt.dayofyear / 365)

# Lags
for lag in [1, 2, 3]:
    weekly_df[f'prop_severos_lag{lag}'] = weekly_df.groupby('uf')['prop_severos'].shift(lag)

# Estatísticas
weekly_df['prop_severos_ma3'] = weekly_df.groupby('uf')['prop_severos'].rolling(3).mean().reset_index(0, drop=True)
weekly_df['prop_severos_tendencia'] = weekly_df.groupby('uf')['prop_severos'].diff()
weekly_df['prop_severos_volatilidade'] = weekly_df.groupby('uf')['prop_severos'].rolling(3).std().reset_index(0, drop=True)

print("✅ Features criadas!")
print(f"   Total: {len(weekly_df.columns)} colunas")

---

## 🎯 Passo 7: Preparação dos Dados

Criamos features baseadas em contagens e características temporais dos acidentes.


---

## 🏗️ Passo 8: Construção do Modelo LSTM

**Arquitetura Otimizada para Séries Temporais:**
- 2 camadas LSTM (128 → 64 neurônios) - captura padrões temporais complexos
- 2 camadas Dense (32 → 16 neurônios) - processamento não-linear
- Output (1 neurônio, linear) - previsão contínua
- Loss: MAE (Mean Absolute Error)
- Métricas: MAE, MSE


In [ ]:
from sklearn.preprocessing import MinMaxScaler

print("🔢 Preparando sequências para LSTM - Previsão de Número de Acidentes...")

# Criar features baseadas em CONTAGEM de acidentes
df_features = weekly_df.set_index('data_inversa').sort_index()

# Features: total de acidentes + características temporais + lags
df_features['total_acidentes_norm'] = df_features['total_acidentes'] / df_features['total_acidentes'].max()

# Lags do número de acidentes (últimas 3 semanas)
for lag in [1, 2, 3]:
    df_features[f'acidentes_lag{lag}'] = df_features.groupby(level=0)['total_acidentes'].shift(lag)

# Média móvel de 3 semanas
df_features['acidentes_ma3'] = df_features['total_acidentes'].rolling(window=3, min_periods=1).mean()

# Tendência (diferença semana atual vs anterior)
df_features['acidentes_tendencia'] = df_features['total_acidentes'].diff()

# Volatilidade (desvio padrão móvel 3 semanas)
df_features['acidentes_volatilidade'] = df_features['total_acidentes'].rolling(window=3, min_periods=1).std()

features_colunas = [
    'total_acidentes', 'pessoas_media', 'veiculos_media', 'fim_semana',
    'sazonalidade_sen', 'sazonalidade_cos',
    'acidentes_lag1', 'acidentes_lag2', 'acidentes_lag3',
    'acidentes_ma3', 'acidentes_tendencia', 'acidentes_volatilidade'
]

df_features = df_features[features_colunas].copy()
df_features = df_features.dropna()

# Target: número total de acidentes
target_values = df_features['total_acidentes'].values

print("\n📊 Estatísticas do target (número total de acidentes por semana):")
print(f"   Min: {target_values.min():.0f} acidentes")
print(f"   Max: {target_values.max():.0f} acidentes")
print(f"   Média: {target_values.mean():.1f} acidentes")
print(f"   Mediana: {np.median(target_values):.0f} acidentes")
print(f"   Desvio padrão: {target_values.std():.1f}")

# Separar features de target
features_sem_target = [col for col in features_colunas if col != 'total_acidentes']
target_col = 'total_acidentes'

# Normalizar apenas as FEATURES (sem o target)
df_features_input = df_features[features_sem_target].copy()
scaler_features = MinMaxScaler(feature_range=(0, 1))
features_scaled = scaler_features.fit_transform(df_features_input.values)

# TARGET também normalizado (para melhor treinamento da rede)
scaler_target = MinMaxScaler(feature_range=(0, 1))
target_scaled = scaler_target.fit_transform(target_values.reshape(-1, 1)).flatten()

n_passos_para_tras = 8
n_features = len(features_sem_target)

X, y = [], []
for i in range(n_passos_para_tras, len(features_scaled)):
    X.append(features_scaled[i-n_passos_para_tras:i, :])
    y.append(target_scaled[i])

X, y = np.array(X), np.array(y)

print(f"\n✅ Sequências criadas para previsão de acidentes!")
print(f"   Shape X: {X.shape}")
print(f"   Shape y: {y.shape}")
print(f"   Range de y normalizado: [{y.min():.3f}, {y.max():.3f}]")

In [ ]:
print("🎯 Dividindo dados temporalmente (respeitando ordem)...")

# Dividir dados temporalmente (85% treino, 15% validação)
split_index = int(len(X) * 0.85)
X_train, X_val = X[:split_index], X[split_index:]
y_train, y_val = y[:split_index], y[split_index:]

# Desnormalizar para mostrar estatísticas reais
y_train_real = scaler_target.inverse_transform(y_train.reshape(-1, 1)).flatten()
y_val_real = scaler_target.inverse_transform(y_val.reshape(-1, 1)).flatten()

print(f"\n📊 Divisão temporal:")
print(f"   Treino: {len(X_train)} sequências ({len(X_train)/len(X)*100:.1f}%)")
print(f"   Validação: {len(X_val)} sequências ({len(X_val)/len(X)*100:.1f}%)")

print(f"\n📊 Estatísticas de y_train (número de acidentes):")
print(f"   Min: {y_train_real.min():.0f} acidentes")
print(f"   Max: {y_train_real.max():.0f} acidentes")
print(f"   Média: {y_train_real.mean():.1f} acidentes/semana")
print(f"   Desvio: {y_train_real.std():.1f}")

print(f"\n📊 Estatísticas de y_val (número de acidentes):")
print(f"   Min: {y_val_real.min():.0f} acidentes")
print(f"   Max: {y_val_real.max():.0f} acidentes")
print(f"   Média: {y_val_real.mean():.1f} acidentes/semana")
print(f"   Desvio: {y_val_real.std():.1f}")

print("\n✅ Dados preparados para previsão de acidentes!")

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

print("🏗️  Construindo modelo LSTM para Previsão de Acidentes...")

# MODELO LSTM OTIMIZADO PARA SÉRIES TEMPORAIS
model = Sequential([
    LSTM(units=128, return_sequences=True, input_shape=(n_passos_para_tras, n_features)),
    Dropout(0.2),
    LSTM(units=64, return_sequences=False),
    Dropout(0.2),
    Dense(units=32, activation='relu'),
    Dropout(0.2),
    Dense(units=16, activation='relu'),
    Dense(units=1, activation='linear')  # Output: valor contínuo
])

# OPTIMIZER
optimizer = Adam(learning_rate=0.001)
model.compile(
    optimizer=optimizer,
    loss='mean_absolute_error',  # MAE: erro médio em número de acidentes
    metrics=['mae', 'mse']
)

print("✅ Modelo construído!")
print("   Arquitetura: LSTM 128 → LSTM 64 → Dense 32 → Dense 16 → Linear 1")
print("   Dropout: 0.2 (em todas as camadas)")
print("   Loss: MAE (Mean Absolute Error)")
print("   Métricas: MAE, MSE")
print("   Learning rate: 0.001")
model.summary()

# CALLBACKS
early_stop = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=0.00001, verbose=1)

print("\n✅ Callbacks configurados!")
print("   Early Stopping: patience=20")
print("   Reduce LR: patience=10")

---

## 🚀 Passo 9: Treinamento do Modelo

Treinamos o modelo com class weights para corrigir desbalanceamento. Monitoramos val_loss para evitar overfitting.


In [ ]:
print("\n" + "="*70)
print("🚀 TREINANDO MODELO LSTM")
print("="*70)
print("\n📊 Prevendo número total de acidentes por semana")
print("⏱️  Aguarde 10-20 minutos...\n")

history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=16,
    validation_data=(X_val, y_val),
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

print("\n" + "="*70)
print("✅ TREINAMENTO CONCLUÍDO!")
print("="*70)

---

## 🔧 Abordagem de Modelagem

### Target: Número Total de Acidentes

Escolhemos prever **contagem de acidentes** porque:

✅ **Padrões temporais claros** - Variação entre dias úteis, fins de semana, sazonalidade  
✅ **Features históricas são preditivas** - Lags e médias móveis capturam tendências  
✅ **LSTM ideal para séries temporais** - Captura dependências de longo prazo  
✅ **Aplicação prática direta** - Alocação de recursos (patrulhas, ambulâncias)

### Vantagens sobre Classificação

Após testes com classificação (que não funcionou bem):

1. **Valores contínuos** - Prever 25 vs 30 acidentes é mais útil que categorias
2. **Métricas intuitivas** - MAE em número de acidentes (ex: ±5 acidentes/semana)
3. **Variabilidade real** - Range maior permite modelo aprender melhor
4. **Honestidade científica** - Mostra o que dados permitem prever

### Arquitetura

- **Input**: Sequências de 8 semanas × 11 features
- **LSTM**: 2 camadas (128 → 64) - captura padrões temporais complexos
- **Dense**: 2 camadas (32 → 16) - processamento não-linear
- **Output**: 1 neurônio linear - número de acidentes
- **Loss**: MAE (Mean Absolute Error)

---



---

## 📊 Passo 10: Avaliação e Métricas

Avaliamos o modelo com MAE (erro médio em acidentes), RMSE, MAPE e R². Comparamos com baseline (sempre prever média).


---

## 📈 Passo 11: Visualizações

Geramos 6 gráficos: curvas de aprendizagem (MAE/MSE), scatter plot real vs previsto, série temporal, distribuição de erros e análise de resíduos.


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

print("📊 Avaliando modelo - Previsão de Acidentes...")

# Predições (normalizadas)
y_pred_norm = model.predict(X_val, verbose=0).flatten()

# Desnormalizar predições e valores reais
y_pred = scaler_target.inverse_transform(y_pred_norm.reshape(-1, 1)).flatten()
y_val_real = scaler_target.inverse_transform(y_val.reshape(-1, 1)).flatten()
y_train_real = scaler_target.inverse_transform(y_train.reshape(-1, 1)).flatten()

# Métricas em escala real (número de acidentes)
mae = mean_absolute_error(y_val_real, y_pred)
mse = mean_squared_error(y_val_real, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_val_real, y_pred)

# MAPE (Mean Absolute Percentage Error)
mape = np.mean(np.abs((y_val_real - y_pred) / (y_val_real + 1))) * 100  # +1 para evitar divisão por zero

print("\n" + "="*70)
print("🎯 RESULTADOS FINAIS - Previsão de Número de Acidentes")
print("="*70)

print(f"\n📊 Métricas de Erro:")
print(f"   MAE (Mean Absolute Error): {mae:.2f} acidentes")
print(f"   RMSE (Root Mean Squared Error): {rmse:.2f} acidentes")
print(f"   MAPE (Mean Absolute Percentage Error): {mape:.2f}%")
print(f"   R² Score: {r2:.4f}")

# Baseline: sempre prever a média
baseline_pred = np.full_like(y_val_real, y_train_real.mean())
baseline_mae = mean_absolute_error(y_val_real, baseline_pred)
baseline_r2 = r2_score(y_val_real, baseline_pred)

print(f"\n📊 Comparação com Baseline (sempre prever média de {y_train_real.mean():.1f} acidentes):")
print(f"   Baseline MAE: {baseline_mae:.2f} acidentes")
print(f"   Baseline R²: {baseline_r2:.4f}")
print(f"   Nosso modelo MAE: {mae:.2f} acidentes ✅")
print(f"   Nosso modelo R²: {r2:.4f} ✅")
print(f"   Melhoria: {((baseline_mae - mae)/baseline_mae*100):.1f}%")

# Estatísticas dos erros
errors = np.abs(y_val_real - y_pred)
print(f"\n📊 Distribuição dos Erros Absolutos:")
print(f"   Mínimo: {errors.min():.2f} acidentes")
print(f"   Máximo: {errors.max():.2f} acidentes")
print(f"   Mediana: {np.median(errors):.2f} acidentes")
print(f"   75º percentil: {np.percentile(errors, 75):.2f} acidentes")

# Percentual de predições com erro < 10 acidentes
erro_threshold = 10
pct_boas = (errors < erro_threshold).sum() / len(errors) * 100
print(f"\n📊 Qualidade das Predições:")
print(f"   {pct_boas:.1f}% das predições têm erro < {erro_threshold} acidentes")

print("\n" + "="*70)

---

## 💾 Passo 12: Salvamento do Modelo

Salvamos o modelo treinado no formato `.keras` para uso futuro.


In [ ]:
plt.figure(figsize=(16, 12))

# 1. Loss (MAE)
plt.subplot(3, 2, 1)
plt.plot(history.history['loss'], label='Treino', color='blue', linewidth=2)
plt.plot(history.history['val_loss'], label='Validação', color='red', linewidth=2)
plt.title('Curvas de Aprendizagem - MAE', fontsize=14, fontweight='bold')
plt.xlabel('Épocas')
plt.ylabel('MAE')
plt.legend()
plt.grid(True, alpha=0.3)

# 2. MSE
plt.subplot(3, 2, 2)
plt.plot(history.history['mse'], label='Treino', color='blue', linewidth=2)
plt.plot(history.history['val_mse'], label='Validação', color='red', linewidth=2)
plt.title('Curvas de Aprendizagem - MSE', fontsize=14, fontweight='bold')
plt.xlabel('Épocas')
plt.ylabel('MSE')
plt.legend()
plt.grid(True, alpha=0.3)

# 3. Real vs Previsto (Scatter Plot)
plt.subplot(3, 2, 3)
plt.scatter(y_val_real, y_pred, alpha=0.5, s=30)
plt.plot([y_val_real.min(), y_val_real.max()], [y_val_real.min(), y_val_real.max()], 'r--', lw=2, label='Linha Perfeita')
plt.title('Real vs Previsto', fontsize=14, fontweight='bold')
plt.xlabel('Número Real de Acidentes')
plt.ylabel('Número Previsto de Acidentes')
plt.legend()
plt.grid(True, alpha=0.3)

# 4. Série Temporal: Real vs Previsto
plt.subplot(3, 2, 4)
plt.plot(y_val_real, label='Real', linewidth=2, alpha=0.7, marker='o', markersize=4)
plt.plot(y_pred, label='Previsto', linewidth=2, alpha=0.7, linestyle='--', marker='x', markersize=4)
plt.title('Comparação Temporal', fontsize=14, fontweight='bold')
plt.xlabel('Amostras')
plt.ylabel('Número de Acidentes')
plt.legend()
plt.grid(True, alpha=0.3)

# 5. Distribuição dos Erros
plt.subplot(3, 2, 5)
errors = y_val_real - y_pred
plt.hist(errors, bins=30, edgecolor='black', alpha=0.7, color='steelblue')
plt.axvline(x=0, color='r', linestyle='--', linewidth=2, label='Erro Zero')
plt.title('Distribuição dos Erros (Resíduos)', fontsize=14, fontweight='bold')
plt.xlabel('Erro (Real - Previsto) [acidentes]')
plt.ylabel('Frequência')
plt.legend()
plt.grid(True, alpha=0.3)

# 6. Resíduos vs Valores Previstos
plt.subplot(3, 2, 6)
plt.scatter(y_pred, errors, alpha=0.5, s=30, color='coral')
plt.axhline(y=0, color='r', linestyle='--', linewidth=2)
plt.title('Resíduos vs Valores Previstos', fontsize=14, fontweight='bold')
plt.xlabel('Número Previsto de Acidentes')
plt.ylabel('Resíduo (Real - Previsto) [acidentes]')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✅ Visualizações geradas!")

In [ ]:
model_filename = 'modelo_lstm_classificacao_risco.keras'
model.save(model_filename)
print(f"💾 Modelo salvo: '{model_filename}'")
print("\n✅ Projeto concluído!")